In [2]:
import tensorflow as tf
import keras

In [3]:
model = keras.saving.load_model("weights-best.weights.keras")
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 120)            │           360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 80)             │         9,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 60)             │         4,860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            61 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,885 (175.34 KB)

 Trainable params: 14,961 (58.44 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 29,924 (116.89 KB)

In [35]:
for layer in model.layers:
    if hasattr(layer, 'activation'):
        print(layer.activation)

<function tanh at 0x000001F2BAC44F40>
<function tanh at 0x000001F2BAC44F40>
<function relu at 0x000001F2BA9C89A0>
<function linear at 0x000001F2BAC45260>


In [4]:
input_length = model.input_shape[1]
dense_layers_info = []
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Dense):
        num_neurons = layer.units
        dense_layers_info.append((num_neurons))

num_dense_layers = len(dense_layers_info)
print("dense layer number:", num_dense_layers)
print("neuron number:", dense_layers_info)

dense layer number: 4
neuron number: [120, 80, 60, 1]


In [5]:
has_input_normalization = isinstance(model.layers[0], keras.src.layers.preprocessing.normalization.Normalization)
has_output_normalization = isinstance(model.layers[-1], keras.src.layers.preprocessing.normalization.Normalization)

In [75]:
with open('variable.txt', 'w') as f:
    f.write(f'''FUNCTION_BLOCK FB_NN_Forward_Propagation
// calculation the forward propagation of neural network
// include normalization layer and denormalization layer
VAR_INPUT
	// input : [x, tau, v] at time k
	input : ARRAY[0..{input_length-1}] OF LREAL;
END_VAR
VAR_OUTPUT
	// output [delta_v] at time k+1
	output : LREAL;
END_VAR
VAR
	// activation function
    activation : FB_Relu;
	// pre-allocation of layer outputs''')
    for i in range(num_dense_layers-1):
        f.write(f'''\n\tHiddenLayer{i+1}_output : ARRAY[0..{dense_layers_info[i]-1}] OF LREAL;''')
    if has_input_normalization:
        f.write(f'''\n\t// input normalizer
normalizer : FB_Normalization_Input;''')
    if has_output_normalization:    
        f.write(f'''\n\t// output denormalizer
	denormalizer : FB_Denormalization_Output;''')
    f.write(f'''\n\t// normalized input
    input_norm : ARRAY[0..{input_length-1}] OF LREAL;	
	// normalized output
	output_norm : LREAL;
	// counters''')
    for idx in range(num_dense_layers-1):
        letter = chr(ord('i') + idx)
        declaration = f"\n\t{letter} : INT;"
        f.write(declaration)
    f.write(f'''\nEND_VAR''')

In [76]:
globalvariable_name = 'GVL_Param_NN'

with open('main.txt', 'w') as f:
    
    if has_input_normalization:
        f.write(f'''// input normalization layer
normalizer(input:=input, input_norm=>input_norm);''')
    else:
        f.write(f'''input_norm :=input;''')
         
    f.write(f'''\n\n// first hidden layer
FOR i:=0 TO {dense_layers_info[0]-1} DO
    HiddenLayer1_output[i] := 0;
    FOR j := 0 TO {input_length-1} DO
        HiddenLayer1_output[i] := HiddenLayer1_output[i] + input_norm[j]*{globalvariable_name}.HiddenLayer1_Weight[i,j];
    END_FOR
    HiddenLayer1_output[i] := HiddenLayer1_output[i] + {globalvariable_name}.HiddenLayer1_Bias[i];
    activation(x:= HiddenLayer1_output[i], result => HiddenLayer1_output[i]);
END_FOR

// second hidden layer''')
    
    for i in range(1, num_dense_layers-1):
        f.write(f'''\nFOR i:=0 TO {dense_layers_info[i]-1} DO
    HiddenLayer{i+1}_output[i] := 0;
    FOR j := 0 TO {dense_layers_info[i-1]-1} DO
        HiddenLayer{i+1}_output[i] := HiddenLayer{i+1}_output[i] + HiddenLayer{i}_output[j]*{globalvariable_name}.HiddenLayer{i+1}_Weight[i,j];
    END_FOR
    HiddenLayer{i+1}_output[i] := HiddenLayer{i+1}_output[i] + {globalvariable_name}.HiddenLayer{i+1}_Bias[i];
    activation(x:= HiddenLayer{i+1}_output[i], result => HiddenLayer{i+1}_output[i]);
END_FOR
''')
        
    f.write(f'''\n// linear output layer\noutput_norm := 0;
FOR j := 0 TO {dense_layers_info[-2]-1} DO
    output_norm := output_norm + HiddenLayer{num_dense_layers-1}_output[j]* {globalvariable_name}.OutputLayer_Weight[j];
END_FOR
output_norm := output_norm + {globalvariable_name}.OutputLayer_Bias;''')

    if has_output_normalization:
         f.write(f'''\n\n// output (de-)normalization layer
denormalizer(output_norm :=output_norm, output => output);''')
    else:
        f.write(f'''\n\noutput :=output_norm;''')
         

In [77]:
import numpy as np
input = np.array([0,2])
input = tf.reshape(input,(1,2))
model.predict(input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


array([[4.075251e-06]], dtype=float32)

In [63]:
intermediate_layer_model = model(inputs=model.input, outputs=model.get_layer("dense_7").output)
intermediate_output = intermediate_layer_model.predict(input)

ValueError: The layer sequential_2 has never been called and thus has no defined input.

In [78]:
import pyads
kk  = model.get_layer("dense_7").get_weights()[0]
data = kk.flatten()
data_to_send = data
plc_var_name = "GVL_Param_NN.HiddenLayer2_Weight"
data_list = data_to_send.tolist()
plc = pyads.Connection('127.0.0.1.1.1', pyads.PORT_TC3PLC1)
plc.open()
plc.write_by_name(plc_var_name,data_list, pyads.PLCTYPE_LREAL*len(data_list))
plc.close()

In [79]:
import pandas as pd
import pyads
document_name = ['norm_mean_out.csv','norm_std_out.csv','norm_mean_in.csv','norm_std_in.csv',
                 'layer1_bias.csv','layer1_weight.csv','layer2_bias.csv','layer2_weight.csv','layer3_bias.csv','layer3_weight.csv']
variable_name = ['GVL_Param_NN.norm_mean_out','GVL_Param_NN.norm_std_out','GVL_Param_NN.norm_mean_in','GVL_Param_NN.norm_std_in',
                 'GVL_Param_NN.HiddenLayer1_Bias','GVL_Param_NN.HiddenLayer1_Weight','GVL_Param_NN.HiddenLayer2_Bias','GVL_Param_NN.HiddenLayer2_Weight',
                 'GVL_Param_NN.OutputLayer_Bias','GVL_Param_NN.OutputLayer_Weight']
for i in range(len(variable_name)):
    data = pd.read_csv(document_name[i],header=None)
    
    data = data.to_numpy().flatten()
    data_to_send = data
    plc_var_name = variable_name[i]
    data_list = data_to_send.tolist()
    plc = pyads.Connection('127.0.0.1.1.1', pyads.PORT_TC3PLC1)
    plc.open()
    plc.write_by_name(plc_var_name,data_list, pyads.PLCTYPE_LREAL*len(data_list))
    plc.close()

FileNotFoundError: [Errno 2] No such file or directory: 'norm_mean_out.csv'

In [57]:
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Dense):
        num_neurons = layer.units
weights, biases = model.layers[1].get_weights()
weights.shape
data = weights
data.append(biases)

AttributeError: 'numpy.ndarray' object has no attribute 'append'